In [33]:
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import pandas as pd
import datetime
from serializable_fairprices_pb2 import FairPrice, FairPrices
import numpy as np
from typing import Union

In [34]:
reader = DataFileReader(open("data/test_intraday_fairprice_service.avro", "rb"), DatumReader())
records = [record for record in reader]
df = pd.DataFrame(records)

In [35]:
df= df.rename(columns={"timestamp": "timestamp_ms"})
df["key"] = df["key"].astype(str)

In [36]:
# Function to decode Protobuf bytes
def decode_protobuf(byte_data):
    fair_prices_proto = FairPrices()  # Instantiate the Protobuf message class
    fair_prices_proto.ParseFromString(byte_data)  # Parse the bytes into the Protobuf message
    fair_price_proto = fair_prices_proto.products[0]
    return fair_price_proto

def extract_field(fair_price: FairPrice, field_name: str) -> Union[int,str]:
    try:
        field_value = getattr(fair_price, field_name)
        return field_value
    except:
        return None

In [37]:
df["fair_price_proto"] = df["value"].apply(decode_protobuf)

In [38]:
fair_price_example = df["fair_price_proto"].iloc[0]
fields_list = [field.name for field in fair_price_example.DESCRIPTOR.fields]

for field_name in fields_list:
    df[field_name] = df["fair_price_proto"].apply(lambda x: extract_field(x,field_name))


In [39]:
for column in df.columns:
    if "_ms" in column:
        df[column] = pd.to_datetime(df[column], unit="ms", utc=True)

In [41]:
df.columns

Index(['key', 'value', 'topic', 'partition', 'offset', 'timestamp_ms',
       'fair_price_proto', 'delivery_area_id', 'delivery_start_ms_utc',
       'delivery_end_ms_utc', 'legacy_avg_bid', 'legacy_best_bid',
       'legacy_fair_price', 'legacy_best_ask', 'legacy_avg_ask', 'max_5mw_bid',
       'best_bid', 'fair_price', 'best_ask', 'max_5mw_ask', 'status',
       'source'],
      dtype='object')

In [50]:
df["time_delta_trade_delivery"] = df["delivery_start_ms_utc"] - df["timestamp_ms"]

In [51]:
df

,key,value,topic,partition,offset,timestamp_ms,fair_price_proto,delivery_area_id,delivery_start_ms_utc,delivery_end_ms_utc,...,legacy_best_ask,legacy_avg_ask,max_5mw_bid,best_bid,fair_price,best_ask,max_5mw_ask,status,source,time_delta_trade_delivery
0,10YNO-2--------T_60,b'\x08\xfb\xb3\x8d\xc5\xd91\x12@\n\x1010YNO-2-...,intraday_FairPriceService.evt,90,178715969,2024-02-11 14:47:38.747000+00:00,"delivery_area_id: ""10YNO-2--------T""\ndelivery...",10YNO-2--------T,2024-02-11 16:00:00+00:00,2024-02-11 17:00:00+00:00,...,7242,7245,7210,7212,7228,7242,7252,1,,0 days 01:12:21.253000
1,10YNO-2--------T_60,b'\x08\xa9\xb4\x8d\xc5\xd91\x12@\n\x1010YNO-2-...,intraday_FairPriceService.evt,90,178715970,2024-02-11 14:47:38.794000+00:00,"delivery_area_id: ""10YNO-2--------T""\ndelivery...",10YNO-2--------T,2024-02-11 17:00:00+00:00,2024-02-11 18:00:00+00:00,...,7909,7909,7823,7832,7866,7909,7909,1,,0 days 02:12:21.206000
2,10YNO-2--------T_60,b'\x08\xaa\xb4\x8d\xc5\xd91\x12@\n\x1010YNO-2-...,intraday_FairPriceService.evt,90,178715971,2024-02-11 14:47:38.794000+00:00,"delivery_area_id: ""10YNO-2--------T""\ndelivery...",10YNO-2--------T,2024-02-11 17:00:00+00:00,2024-02-11 18:00:00+00:00,...,7909,7909,7823,7832,7866,7909,7909,1,,0 days 02:12:21.206000
3,10YNO-2--------T_60,b'\x08\xb1\xb4\x8d\xc5\xd91\x12@\n\x1010YNO-2-...,intraday_FairPriceService.evt,90,178715972,2024-02-11 14:47:38.801000+00:00,"delivery_area_id: ""10YNO-2--------T""\ndelivery...",10YNO-2--------T,2024-02-11 17:00:00+00:00,2024-02-11 18:00:00+00:00,...,7909,7909,7833,7833,7871,7909,7909,1,,0 days 02:12:21.199000
4,10YNO-2--------T_60,b'\x08\xe1\xb4\x8d\xc5\xd91\x12@\n\x1010YNO-2-...,intraday_FairPriceService.evt,90,178715973,2024-02-11 14:47:38.849000+00:00,"delivery_area_id: ""10YNO-2--------T""\ndelivery...",10YNO-2--------T,2024-02-11 17:00:00+00:00,2024-02-11 18:00:00+00:00,...,7908,7908,7833,7833,7870,7908,7908,1,,0 days 02:12:21.151000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130906,10YDE-EON------1_60,b'\x08\xdd\xee\xad\xc3\xd71\x12@\n\x1010YDE-EO...,intraday_FairPriceService.evt,96,481109560,2024-02-05 08:38:45.343000+00:00,"delivery_area_id: ""10YDE-EON------1""\ndelivery...",10YDE-EON------1,2024-02-05 12:00:00+00:00,2024-02-05 13:00:00+00:00,...,2170,2187,2117,2120,2152,2170,2200,1,,0 days 03:21:14.657000
130907,10YDE-EON------1_60,b'\x08\xf1\xee\xad\xc3\xd71\x12@\n\x1010YDE-EO...,intraday_FairPriceService.evt,96,481109561,2024-02-05 08:38:45.363000+00:00,"delivery_area_id: ""10YDE-EON------1""\ndelivery...",10YDE-EON------1,2024-02-05 12:00:00+00:00,2024-02-05 13:00:00+00:00,...,2170,2187,2120,2130,2156,2170,2200,1,,0 days 03:21:14.637000
130908,10YDE-EON------1_60,b'\x08\xfb\xee\xad\xc3\xd71\x12@\n\x1010YDE-EO...,intraday_FairPriceService.evt,96,481109562,2024-02-05 08:38:45.373000+00:00,"delivery_area_id: ""10YDE-EON------1""\ndelivery...",10YDE-EON------1,2024-02-05 11:00:00+00:00,2024-02-05 12:00:00+00:00,...,3200,3202,3152,3153,3177,3200,3209,1,,0 days 02:21:14.627000
130909,10YNO-1--------2_60,b'\x08\xfe\xee\xad\xc3\xd71\x12@\n\x1010YNO-1-...,intraday_FairPriceService.evt,96,481109563,2024-02-05 08:38:45.374000+00:00,"delivery_area_id: ""10YNO-1--------2""\ndelivery...",10YNO-1--------2,2024-02-05 12:00:00+00:00,2024-02-05 13:00:00+00:00,...,5616,5620,5437,5441,5529,5616,5621,1,,0 days 03:21:14.626000
